In [ ]:
"""
Part 1: Tree-Based Models with Cost-Sensitive Learning
========================================================
Models: XGBoost, LightGBM, Random Forest, Gradient Boosting, Stacking, Voting
Runtime: 15-20 minutes on CPU
Hardware: Any (CPU/GPU doesn't matter for tree models)

Model Rationale:
- Tree-based models excel at tabular healthcare data with mixed features
- Cost-sensitive learning accounts for 35:1 FN:FP cost ratio ($17.5K vs $500)
- Calibration improves probability reliability for clinical decision-making
- Ensemble methods combine diverse model strengths
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score, precision_score, recall_score, f1_score, brier_score_loss, confusion_matrix
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from imblearn.over_sampling import SMOTE
import joblib
import json
import time
import warnings
warnings.filterwarnings('ignore')
from google.colab import drive

drive.mount('/content/drive')
# Check that the mount worked
!ls /content/drive/MyDrive



In [ ]:
# ============================================================================
# CONFIGURATION
# ============================================================================

COST_FP = 500       # Cost of unnecessary intervention
COST_FN = 17500     # Cost of missed readmission (35x higher)
WEIGHT_RATIO = COST_FN / COST_FP

# ============================================================================
# UTILITY FUNCTIONS
# ============================================================================

def calculate_cost(y_true, y_pred):
    """Calculate total business cost from predictions"""
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return fp * COST_FP + fn * COST_FN

def find_optimal_threshold(y_true, y_proba):
    """Find probability threshold that minimizes business cost"""
    thresholds = np.linspace(0.1, 0.9, 100)
    costs = [calculate_cost(y_true, (y_proba >= t).astype(int)) for t in thresholds]
    return thresholds[np.argmin(costs)]

def evaluate_model(name, y_true, y_pred, y_proba, threshold):
    """Return comprehensive metrics dictionary"""
    return {
        'model': name,
        'auc_roc': roc_auc_score(y_true, y_proba),
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred),
        'recall': recall_score(y_true, y_pred),
        'f1': f1_score(y_true, y_pred),
        'brier_score': brier_score_loss(y_true, y_proba),
        'optimal_threshold': threshold,
        'business_cost': calculate_cost(y_true, y_pred)
    }



In [ ]:
# ============================================================================
# DATA PREPARATION
# ============================================================================

print("Loading and preparing data...")

df = pd.read_csv("/content/drive/MyDrive/BerkeleyML/CapstoneHospitalReadmission/eda/processed_data.csv")

X = df.drop('readmitted_binary', axis=1)
y = df['readmitted_binary']

# Train-test split with stratification to maintain class balance
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Standardization (important for calibration and ensembles)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# SMOTE with conservative ratio (0.7 instead of 1.0) for better generalization
smote = SMOTE(random_state=42, sampling_strategy=0.7)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_scaled, y_train)

# Sample weights for cost-sensitive learning
sample_weights = np.where(y_train_balanced == 1, WEIGHT_RATIO, 1.0)

joblib.dump(scaler, "/content/drive/MyDrive/BerkeleyML/CapstoneHospitalReadmission/modelsf/scaler_final.pkl")
print(f"Data ready: {X_train_balanced.shape[0]:,} train, {X_test.shape[0]:,} test")

Loading and preparing data...
Data ready: 122,954 train, 20,354 test


In [ ]:
# ============================================================================
# MODEL 1: XGBOOST
# ============================================================================
# Why XGBoost: Handles mixed features well, built-in regularization,
# scale_pos_weight parameter perfect for cost-sensitive learning

print("\n[1/6] Training XGBoost...")
start = time.time()

xgb_params = {
    'n_estimators': [200, 300, 400],
    'max_depth': [4, 6, 8],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.8, 0.9],
    'colsample_bytree': [0.8, 0.9],
    'min_child_weight': [1, 3, 5],
    'gamma': [0, 0.1, 0.2],
    'reg_alpha': [0, 0.1, 0.5],
    'reg_lambda': [1, 1.5, 2]
}

xgb_model = RandomizedSearchCV(
    XGBClassifier(random_state=42, eval_metric='logloss', scale_pos_weight=WEIGHT_RATIO, use_label_encoder=False),
    xgb_params, n_iter=30, cv=5, scoring='roc_auc', n_jobs=-1, random_state=42, verbose=0
).fit(X_train_balanced, y_train_balanced, sample_weight=sample_weights)

# Isotonic calibration: Non-parametric, works well for tree models
xgb_calibrated = CalibratedClassifierCV(xgb_model.best_estimator_, method='isotonic', cv=3)
xgb_calibrated.fit(X_train_balanced, y_train_balanced)

y_proba_xgb = xgb_calibrated.predict_proba(X_test_scaled)[:, 1]
thresh_xgb = find_optimal_threshold(y_test, y_proba_xgb)
y_pred_xgb = (y_proba_xgb >= thresh_xgb).astype(int)

xgb_metrics = evaluate_model('XGBoost', y_test, y_pred_xgb, y_proba_xgb, thresh_xgb)
joblib.dump(xgb_calibrated, "/content/drive/MyDrive/BerkeleyML/CapstoneHospitalReadmission/modelsf/xgboost_calibrated_final.pkl")
print(f"  AUC: {xgb_metrics['auc_roc']:.4f} | Time: {time.time()-start:.1f}s")




[1/6] Training XGBoost...
  AUC: 0.5865 | Time: 158.3s


In [ ]:
# ============================================================================
# MODEL 2: LIGHTGBM
# ============================================================================
# Why LightGBM: Faster than XGBoost, leaf-wise growth better for deep trees,
# handles categorical features natively, good for large datasets

print("\n[2/6] Training LightGBM...")
start = time.time()

lgbm_params = {
    'n_estimators': [200, 300, 400],
    'max_depth': [5, 7, 9, -1],
    'learning_rate': [0.01, 0.03, 0.05],
    'num_leaves': [31, 50, 70, 100],
    'min_child_samples': [20, 30, 50],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
    'reg_alpha': [0, 0.1, 0.5],
    'reg_lambda': [0, 0.1, 0.5]
}

lgbm_model = RandomizedSearchCV(
    LGBMClassifier(random_state=42, verbose=-1, class_weight={0: 1.0, 1: WEIGHT_RATIO}),
    lgbm_params, n_iter=30, cv=5, scoring='roc_auc', n_jobs=-1, random_state=42, verbose=0
).fit(X_train_balanced, y_train_balanced)

lgbm_calibrated = CalibratedClassifierCV(lgbm_model.best_estimator_, method='isotonic', cv=3)
lgbm_calibrated.fit(X_train_balanced, y_train_balanced)

y_proba_lgbm = lgbm_calibrated.predict_proba(X_test_scaled)[:, 1]
thresh_lgbm = find_optimal_threshold(y_test, y_proba_lgbm)
y_pred_lgbm = (y_proba_lgbm >= thresh_lgbm).astype(int)

lgbm_metrics = evaluate_model('LightGBM', y_test, y_pred_lgbm, y_proba_lgbm, thresh_lgbm)
joblib.dump(lgbm_calibrated, "/content/drive/MyDrive/BerkeleyML/CapstoneHospitalReadmission/modelsf/lightgbm_calibrated_final.pkl")
print(f"  AUC: {lgbm_metrics['auc_roc']:.4f} | Time: {time.time()-start:.1f}s")




[2/6] Training LightGBM...


KeyboardInterrupt: 

In [ ]:
# ============================================================================
# MODEL 3: RANDOM FOREST
# ============================================================================
# Why Random Forest: Robust to overfitting, interpretable feature importance,
# handles non-linear relationships, good baseline ensemble

print("\n[3/6] Training Random Forest...")
start = time.time()

rf_model = RandomForestClassifier(
    n_estimators=300, max_depth=8, min_samples_split=20, min_samples_leaf=10,
    class_weight={0: 1.0, 1: WEIGHT_RATIO}, random_state=42, n_jobs=-1
).fit(X_train_balanced, y_train_balanced)

rf_calibrated = CalibratedClassifierCV(rf_model, method='isotonic', cv=3)
rf_calibrated.fit(X_train_balanced, y_train_balanced)

y_proba_rf = rf_calibrated.predict_proba(X_test_scaled)[:, 1]
thresh_rf = find_optimal_threshold(y_test, y_proba_rf)
y_pred_rf = (y_proba_rf >= thresh_rf).astype(int)

rf_metrics = evaluate_model('Random Forest', y_test, y_pred_rf, y_proba_rf, thresh_rf)
joblib.dump(rf_calibrated, "/content/drive/MyDrive/BerkeleyML/CapstoneHospitalReadmission/modelsf/random_forest_calibrated_final.pkl")
print(f"  AUC: {rf_metrics['auc_roc']:.4f} | Time: {time.time()-start:.1f}s")




[3/6] Training Random Forest...
  AUC: 0.6382 | Time: 16.3s


In [ ]:
# ============================================================================
# MODEL 4: GRADIENT BOOSTING
# ============================================================================
# Why Gradient Boosting: Sequential error correction, smooth decision boundaries,
# often best single model, scikit-learn implementation stable

print("\n[4/6] Training Gradient Boosting...")
start = time.time()

gb_model = GradientBoostingClassifier(
    n_estimators=200, max_depth=5, learning_rate=0.1, subsample=0.8, random_state=42
).fit(X_train_balanced, y_train_balanced, sample_weight=sample_weights)

gb_calibrated = CalibratedClassifierCV(gb_model, method='isotonic', cv=3)
gb_calibrated.fit(X_train_balanced, y_train_balanced)

y_proba_gb = gb_calibrated.predict_proba(X_test_scaled)[:, 1]
thresh_gb = find_optimal_threshold(y_test, y_proba_gb)
y_pred_gb = (y_proba_gb >= thresh_gb).astype(int)

gb_metrics = evaluate_model('Gradient Boosting', y_test, y_pred_gb, y_proba_gb, thresh_gb)
joblib.dump(gb_calibrated, "/content/drive/MyDrive/BerkeleyML/CapstoneHospitalReadmission/modelsf/gradient_boosting_calibrated_final.pkl")
print(f"  AUC: {gb_metrics['auc_roc']:.4f} | Time: {time.time()-start:.1f}s")




[4/6] Training Gradient Boosting...
  AUC: 0.6223 | Time: 363.4s


In [ ]:
# ============================================================================
# MODEL 5: STACKING ENSEMBLE
# ============================================================================
# Why Stacking: Meta-learner learns optimal combination of base models,
# captures complementary strengths, often outperforms individual models

print("\n[5/6] Training Stacking Ensemble...")
start = time.time()

base_estimators = [
    ('xgb', xgb_model.best_estimator_),
    ('lgbm', lgbm_model.best_estimator_),
    ('rf', rf_model),
    ('gb', gb_model)
]

stacking_model = StackingClassifier(
    estimators=base_estimators,
    final_estimator=LogisticRegression(max_iter=1000, class_weight={0: 1.0, 1: WEIGHT_RATIO}, random_state=42),
    cv=5, n_jobs=-1, passthrough=False
).fit(X_train_balanced, y_train_balanced)

# Sigmoid calibration: Better for already-combined probabilities
stacking_calibrated = CalibratedClassifierCV(stacking_model, method='sigmoid', cv=3)
stacking_calibrated.fit(X_train_balanced, y_train_balanced)

y_proba_stack = stacking_calibrated.predict_proba(X_test_scaled)[:, 1]
thresh_stack = find_optimal_threshold(y_test, y_proba_stack)
y_pred_stack = (y_proba_stack >= thresh_stack).astype(int)

stack_metrics = evaluate_model('Stacking Ensemble', y_test, y_pred_stack, y_proba_stack, thresh_stack)
joblib.dump(stacking_calibrated, "/content/drive/MyDrive/BerkeleyML/CapstoneHospitalReadmission/modelsf/stacking_ensemble_final.pkl")
print(f"  AUC: {stack_metrics['auc_roc']:.4f} | Time: {time.time()-start:.1f}s")



In [ ]:
# ============================================================================
# MODEL 6: VOTING ENSEMBLE
# ============================================================================
# Why Voting: Simple probability averaging, more robust than single models,
# reduces variance, good baseline ensemble

print("\n[6/6] Training Voting Ensemble...")
start = time.time()

voting_model = VotingClassifier(
    estimators=base_estimators, voting='soft', n_jobs=-1
).fit(X_train_balanced, y_train_balanced)

voting_calibrated = CalibratedClassifierCV(voting_model, method='sigmoid', cv=3)
voting_calibrated.fit(X_train_balanced, y_train_balanced)

y_proba_vote = voting_calibrated.predict_proba(X_test_scaled)[:, 1]
thresh_vote = find_optimal_threshold(y_test, y_proba_vote)
y_pred_vote = (y_proba_vote >= thresh_vote).astype(int)

vote_metrics = evaluate_model('Voting Ensemble', y_test, y_pred_vote, y_proba_vote, thresh_vote)
joblib.dump(voting_calibrated, "/content/drive/MyDrive/BerkeleyML/CapstoneHospitalReadmission/modelsf/voting_ensemble_final.pkl")
print(f"  AUC: {vote_metrics['auc_roc']:.4f} | Time: {time.time()-start:.1f}s")



In [ ]:
# ============================================================================
# RESULTS & SAVE
# ============================================================================

all_metrics = [xgb_metrics, lgbm_metrics, rf_metrics, gb_metrics, stack_metrics, vote_metrics]
comparison_df = pd.DataFrame(all_metrics).sort_values('auc_roc', ascending=False)

print("\n" + "="*80)
print("TREE-BASED MODELS SUMMARY")
print("="*80)
print(comparison_df[['model', 'auc_roc', 'precision', 'recall', 'f1', 'brier_score']].to_string(index=False))

best = comparison_df.iloc[0]
print(f"\nBest Model: {best['model']} (AUC: {best['auc_roc']:.4f})")

# Save artifacts for Part 2
comparison_df.to_csv('reports/tree_models_comparison.csv', index=False)
np.savez("/content/drive/MyDrive/BerkeleyML/CapstoneHospitalReadmission/modelsf/tree_model_predictions.npz",
         y_test=y_test.values, xgboost=y_proba_xgb, lightgbm=y_proba_lgbm,
         random_forest=y_proba_rf, gradient_boosting=y_proba_gb,
         stacking=y_proba_stack, voting=y_proba_vote)
np.save("/content/drive/MyDrive/BerkeleyML/CapstoneHospitalReadmission/modelsf/test_indices.npy", X_test.index.values)

config = {
    'best_tree_model': best['model'],
    'best_auc': float(best['auc_roc']),
    'cost_false_positive': COST_FP,
    'cost_false_negative': COST_FN,
    'weight_ratio': float(WEIGHT_RATIO),
    'feature_count': X.shape[1]
}
with open("/content/drive/MyDrive/BerkeleyML/CapstoneHospitalReadmission/modelsf/tree_models_config.json", 'w') as f:
    json.dump(config, f, indent=4)

print("\n✓ Part 1 complete. Run Part 2 for neural networks and final ensemble.")